In [65]:

import os
import csv
import re

units = {
    "s": 1.0,
    "ms": 0.001,
    "µs": 0.000001,
    "ns": 0.000000001,
    "bytes": 1.0 / 1048576,  # 1 MB = 1024 * 1024 B
    "kilo_bytes": 1.0 / 1024  # 1 KB = 1024 B
}

batch = 5 # When n is small, the leader MSM occupies a big bottleneck, however, this can be efficiently optimized by parallel the batch MSM.

def removeConsecutiveDuplicates(s):
    if len(s) < 2:
        return s
    if s[0] != s[1] or s[0] != '.':
        return s[0] + removeConsecutiveDuplicates(s[1:])
    return removeConsecutiveDuplicates(s[1:])

def splitString(str):
    alpha = ""
    num = ""
    for i in range(len(str)):
        if (str[i].isdigit() or str[i] == '.'):
            num = num + str[i]
        else:
            alpha += str[i]  
    return alpha, num

In [66]:
# # sumcheck
# data = []

# # Read files
# for filename in os.listdir('sumcheck'):
#     if filename.startswith('sumcheck_') and filename.endswith('.txt'):
#         with open(f'sumcheck/{filename}', 'r') as file:
#             content = file.readlines()

#             record = {'n': None, 'd_time(s)': None, 'd_comm(MB)': None, 'l_time(ms)': None}
#             record['n'] = int(re.findall(r'sumcheck_(\d+)', filename)[0])

#             for line in content:
#                 if 'End:' in line and 'SumcheckProduct' in line:
#                     cleaned_line = removeConsecutiveDuplicates(line)
#                     time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
#                     if time_string:
#                         alpha, num = splitString(time_string.group(0))
#                         time_in_ms = float(num) * units[alpha.strip()]
#                         if 'Distributed' in line:
#                             record['d_time(s)'] = time_in_ms
#                         else:
#                             record['l_time(ms)'] = time_in_ms

#                 if 'Comm:' in line:
#                     comm = sum(map(int, re.findall(r'(\d+), (\d+)', line)[0]))
#                     record['d_comm(MB)'] = comm * units['bytes']

#             data.append(record)

# # Save to .csv
# csv_file = 'sumcheck.csv'
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=['n', 'd_time(s)', 'd_comm(MB)', 'l_time(ms)'])
#     writer.writeheader()
#     for row in data:
#         writer.writerow(row)


In [68]:
# d_Plonk
data = []

d_plonk_operations = [
    ("FFT1", "local"),
    ("Compute r", "local"),
    ("Compute t", "local"),
    ("Local DPP", "local"),
    ("DppRand", "local"),
    ("Division", "local"),
    ("FFT2", "king"),
    ("DPP", "king"),
    ("ComToKing", "comm"),
    ("ComFromKing", "comm"),
    ("Opening shares", "king"),
    ("Packing shares", "king"),
    ("Unpack Pack shares", "king"),
    ("Base MSM", "msm")
]

# Read leader server files
for filename in os.listdir('plonk_d'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'plonk_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l': None, 'd_time(s)': 0, 'd_mem(MB)': 0, 'd_comm(MB)': 0, 'local': 0, 'king': 0, "comm": 0, "msm": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['l'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            
            for line in content:
                for op_name, op_var in d_plonk_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'bytes_sent:' in line:
                    bytes_sent = int(re.search(r'bytes_sent: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_sent * units['bytes']

                if 'bytes_recv:' in line:
                    bytes_recv = int(re.search(r'bytes_recv: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_recv * units['bytes']
                    
                if 'Maximum resident set size' in line:
                    mem = int(re.search(r'(\d+)', line).group(0))
                    record['d_mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time
            record['d_time(s)'] = record['local'] + record['king'] + record['comm'] + record['msm']
            
            data.append(record)

# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'plonk_d.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l', 'd_time(s)', 'd_mem(MB)', 'd_comm(MB)', 'local','king', "comm", "msm"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)
        
data = []

# Read worker server files
for filename in os.listdir('plonk_d'):
    if filename.startswith('log_1_') and filename.endswith('.txt'):
        with open(f'plonk_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l': None, 'd_time(s)': 0, 'd_mem(MB)': 0, 'd_comm(MB)': 0, 'local': 0, 'king': 0, "comm": 0, "msm": 0}
            record['n'] = int(re.findall(r'log_1_(\d+)_(\d+)', filename)[0][0])
            record['l'] = int(re.findall(r'log_1_(\d+)_(\d+)', filename)[0][1])
            
            for line in content:
                for op_name, op_var in d_plonk_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'bytes_sent:' in line:
                    bytes_sent = int(re.search(r'bytes_sent: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_sent * units['bytes']

                if 'bytes_recv:' in line:
                    bytes_recv = int(re.search(r'bytes_recv: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_recv * units['bytes']
                    
                if 'Maximum resident set size' in line:
                    mem = int(re.search(r'(\d+)', line).group(0))
                    record['d_mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time
            record['d_time(s)'] = record['local'] + record['king'] + record['comm'] + record['msm']
            
            data.append(record)

# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'plonk_d_worker.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l', 'd_time(s)', 'd_mem(MB)', 'd_comm(MB)', 'local','king', "comm", "msm"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [69]:

# local_GKR Depth=8
data = []

operations = [
    ("Commit", "poly_comm"),
    ("GKR Round", "sumcheck"),
    ("Open", "poly_comm"),
]

# Read files
for filename in os.listdir('gkr_local'):
    if filename.startswith('gkr_') and filename.endswith('.txt'):
        with open(f'gkr_local/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l_time(s)': 0, 'poly_comm': 0, "sumcheck": 0}
            # +3 for depth 8
            record['n'] = int(re.findall(r'gkr_(\d+)', filename)[0]) + 3
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

            # Calculate total time
            record['l_time(s)'] = (record['sumcheck'] + record['poly_comm']) 
            
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'gkr_local.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l_time(s)', 'sumcheck', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [70]:
# local_Hyperplonk
data = []

operations = [
    ("Commit", "poly_comm"),
    ("Gate identity", "gate"),
    ("Wire identity", "wire"),
    ("Open", "poly_comm"),
]

# Read files
for filename in os.listdir('hyperplonk_local'):
    if filename.startswith('hyperplonk_') and filename.endswith('.txt'):
        with open(f'hyperplonk_local/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l_time(s)': 0, 'poly_comm': 0, "gate": 0, "wire": 0}
            record['n'] = int(re.findall(r'hyperplonk_(\d+)', filename)[0])
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

            # Calculate total time
            record['l_time(s)'] = (record['gate'] + record['wire'] + record['poly_comm']) 
            
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'hyperplonk_local.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l_time(s)', 'gate', 'wire', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [71]:

# d_GKR Depth=8
data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

# Read files
for filename in os.listdir('gkr_d'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'gkr_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l': None, 'd_time(s)': 0, 'd_comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            # +3 for depth 8
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0]) + 3 
            record['l'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['d_comm(MB)'] = comm * units['bytes']


             # Calculate total time
            record['d_time(s)'] = record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch
                       
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'gkr_d.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l', 'd_time(s)', 'd_comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [72]:


# d_Hyperplonk l=32 d=8
data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

# Read files
for filename in os.listdir('hyperplonk_d'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'hyperplonk_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'd_time(s)': 0, 'd_comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['l'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['d_comm(MB)'] = comm * units['bytes']

             # Calculate total time
            record['d_time(s)'] = record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch
            
            data.append(record)

# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = 'hyperplonk_d.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l', 'd_time(s)', 'd_comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [73]:
# d_GKR Depth=8 w=21 varying parties
w = 21
n = w + 3
data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

# Read files
for filename in os.listdir('gkr_d'):
    if filename.startswith(f'log_0_{w}') and filename.endswith('.txt'):
        with open(f'gkr_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'parties': None, 'n': n, 'd_time(s)': 0, 'd_comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            l = int(re.findall(fr'log_0_{w}_(\d+)', filename)[0]) # also k in the paper
            record['parties'] = l * 4 # party = 2^L * 4 where 4 is the packing factor
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['d_comm(MB)'] = comm * units['bytes']

             # Calculate total time
            record['d_time(s)'] = record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch          
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['parties'])

# Save to .csv
csv_file = 'gkr_l.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['parties', 'n', 'd_time(s)', 'd_comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [74]:

# d_HyperPlonk n=24 varying parties
n = 24
data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

# Read files
for filename in os.listdir('hyperplonk_d'):
    if filename.startswith(f'log_0_{n}') and filename.endswith('.txt'):
        with open(f'hyperplonk_d/{filename}', 'r') as file:
            content = file.readlines()

            record = {'parties': None, 'n': n, 'd_time(s)': 0, 'd_comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            l = int(re.findall(fr'log_0_{n}_(\d+)', filename)[0]) # also k in the paper
            record['parties'] = l * 4 # party = 2^L * 4 where 4 is the packing factor
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['d_comm(MB)'] = comm * units['bytes']

             # Calculate total time
            record['d_time(s)'] = record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch        
            data.append(record)

# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['parties'])

# Save to .csv
csv_file = 'hyperplonk_l.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['parties', 'n', 'd_time(s)', 'd_comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)